In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd.function import Function
import os
import pickle
import numpy as np
import pandas as pd
import math
from itertools import chain
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import itertools
import random
import tqdm
from IPython.display import display, HTML
import matplotlib.pyplot as plt
from scipy import signal
from scipy.fft import fftshift

from einops import rearrange, repeat
from einops.layers.torch import Rearrange
from torch import nn, einsum
import math
import logging
from functools import partial
from collections import OrderedDict
from sklearn.metrics import classification_report
from torchsummary import summary

In [ ]:
!nvidia-smi
torch.cuda.is_available()

# Data read and reshape

In [31]:
cd /project/hikaku_db/chen-z/Workspace/

/project/hikaku_db/chen-z/Workspace


In [ ]:
cd /project/hikaku_db/data/sleep_SHHS/stages_sig/

In [12]:
ls

Bal_Trans_8lay_nodrop_mulchan_eegspec_state
Bal_Trans_8lay_nodrop_mulchan_eegspec_state_R6
Balance_set/
PreTest_Trans_8lay_50drop_noW_eegspec_state1
PreTest_Trans_8lay_50drop_noW_eegspec_state2
PreTest_Trans_8lay_50drop_noW_eegspec_state3
PreTest_Trans_8lay_50drop_noW_eegspec_state4
PreTest_Trans_8lay_nodrop_mulchan_eegspec_state1
PreTest_Trans_8lay_nodrop_mulchan_eegspec_state2
PreTest_Trans_8lay_nodrop_mulchan_eegspec_state3
PreTest_Trans_8lay_nodrop_mulchan_eegspec_state4
PreTest_Trans_8lay_nodrop_mulchan_eegspec_state5
PreTest_Trans_8lay_nodrop_mulchan_eegspec_state6
PreTrain_Trans_8lay_nodrop_mulchan_eegspec_state
PreTrained_8lay_nodrop/
PreTrained_t_Trans_8lay_50drop/
PreTrained_t_Trans_8lay_50drop_mulchan_eegspec_stateTrain
Pre_test_Trans_8lay_50drop_mulchan_eegspec_state1
Pre_test_Trans_8lay_50drop_mulchan_eegspec_state2
prelabel.csv
spec_c3_pre.npy
spec_c4_pre.npy


In [3]:
cd /home/is/zheng-ch/Transformer/PreTdat

/home/is/zheng-ch/Transformer/PreTdat


In [4]:
dat1 = np.load('/project/hikaku_db/data/sleep_SHHS/stages_sig/C3_spec_30_np/spec_c3_5007_5793_30.npy')
dat2 = np.load('/project/hikaku_db/data/sleep_SHHS/stages_sig/C4_spec_30_np/spec_5007_5793_30.npy')
print(dat1.shape)
print(dat2.shape)

dat = np.concatenate((dat1.reshape(-1,1,32,30), dat2.reshape(-1,1,32,30)), axis=1)
print(dat.shape)

(786606, 32, 30)
(786606, 32, 30)
(786606, 2, 32, 30)


In [ ]:
index = pd.read_csv("/project/hikaku_db/data/sleep_SHHS/stages_sig/ann_delrecords_5class.csv", header=None)
index = index.astype(int)
print(index[5007610:5794216].apply(pd.value_counts))
label = index[5007610:5794216].values.tolist()#list
print(len(label))

In [ ]:
class Mydatasets(torch.utils.data.Dataset):
    def __init__(self, data,label ,transform = None):
        self.transform = transform

        self.data = data
        self.label = label

        self.datanum = len(data)

    def __len__(self):
        return self.datanum

    def __getitem__(self, idx):
        
        out_data = torch.tensor(self.data[idx]).float()
        out_label = torch.tensor(self.label[idx])
        if self.transform:
            out_data = self.transform(out_data)

        return out_data, out_label

In [ ]:
print(dat.shape)
# label = index[0:100].values.tolist()#list
print(len(label))

fixdata = dat[:,:,0:16,:]
mean_p1 = np.mean(dat[:,:,16:20,:], axis = 2)
mean_p2 = np.mean(dat[:,:,20:24,:], axis = 2)
mean_p3 = np.mean(dat[:,:,24:28,:], axis = 2)
mean_p4 = np.mean(dat[:,:,28:32,:], axis = 2)
num_data = len(dat)

ch = 2
inputdat = np.concatenate((fixdata,mean_p1.reshape(num_data, ch, 1, 30),mean_p2.reshape(num_data, ch, 1, 30),mean_p3.reshape(num_data, ch, 1, 30),mean_p4.reshape(num_data, ch, 1, 30)),axis=2)
print(inputdat.shape)

train, test, train_label, test_label = train_test_split(inputdat, np.array(label),test_size = 0.1,stratify = label,random_state = 0)
print('train data:',len(train))
print('test data:',len(test))

train_data_set = Mydatasets(data=train,label=train_label)
test_data_set = Mydatasets(data=test,label=test_label)

train_dataloader = torch.utils.data.DataLoader(train_data_set, batch_size = 64, shuffle=True)
test_dataloader = torch.utils.data.DataLoader(test_data_set, batch_size = 64, shuffle=False)

# Transformer Model

In [ ]:
##Transformer model, flatted the EEG channel, allows parallel computation the multi-band from multi-channel. 

class PreNorm(nn.Module): 
    def __init__(self, dim, fn):
        super().__init__()
        
        self.norm = nn.LayerNorm(dim)
        self.fn = fn
    
    def forward(self, x, **kwargs):
        return self.fn(self.norm(x), **kwargs)

class FeedForward(nn.Module):
    def __init__(self, dim, hidden_dim, dropout = 0.5):
        super().__init__()
        
        self.net = nn.Sequential(
            nn.Linear(dim, hidden_dim),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, dim),
            nn.Dropout(dropout)
        )
    
    def forward(self, x):
        return self.net(x)

class Attention(nn.Module):
    def __init__(self, dim, heads, dim_head, dropout = 0.5):
        super().__init__()
        
        inner_dim = dim_head *  heads       ##32(4*8)   
        project_out = not (heads == 1 and dim_head == dim)

        self.heads = heads
        self.scale = dim_head ** -0.5

        self.attend = nn.Softmax(dim = -1)
        self.to_qkv = nn.Linear(dim, inner_dim * 3, bias = False)   ##better to dim to dim*3

        self.to_out = nn.Sequential(
            nn.Linear(inner_dim, dim),
            nn.Dropout(dropout)
        ) if project_out else nn.Identity()

    def forward(self, x):
        b, n, _, h = *x.shape, self.heads
        qkv = self.to_qkv(x).chunk(3, dim = -1)
        
        q, k, v = map(lambda t: rearrange(t, 'b n (h d) -> b h n d', h = h), qkv)
        dots = einsum('b h i d, b h j d -> b h i j', q, k) * self.scale

        attn = self.attend(dots)

        out = einsum('b h i j, b h j d -> b h i d', attn, v)
        out = rearrange(out, 'b h n d -> b n (h d)')
        
        return self.to_out(out)

class Transformer(nn.Module):           ##Register the blocks into whole network
    def __init__(self, dim, depth, heads, dim_head, mlp_dim, dropout = 0.5):
        super().__init__()
        
        self.layers = nn.ModuleList([])
        
        for _ in range(depth):
            self.layers.append(nn.ModuleList([
                PreNorm(dim, Attention(dim, heads = heads, dim_head = dim_head, dropout = dropout)),
                PreNorm(dim, FeedForward(dim, mlp_dim, dropout = dropout))
            ]))
    def forward(self, x):
        for attn, ff in self.layers:
            x = attn(x) + x
            x = ff(x) + x
        return x

class ViT(nn.Module):
    def __init__(self, *, image_size, time_size, fre_size, num_classes, dim, depth, heads, mlp_dim, pool = 'cls', channels , dim_head, dropout = 0.5, emb_dropout = 0.):
        super().__init__()
        assert image_size == 30  ##Time dimensions must equal to 30s
        num_patches = 300       #30*5'
        patch_dim = time_size * fre_size    #1*4'
        assert pool in {'cls', 'mean'}, 'pool type must be either cls (cls token) or mean (mean pooling)'

        self.to_patch_embedding = nn.Sequential(
            Rearrange('b c (h p1) (w p2) -> b (h w c) (p1 p2)', p1 = fre_size, p2 = time_size),
            nn.Linear(patch_dim, dim),        ##4 to dim
        )

        self.pos_embedding = nn.Parameter(torch.randn(1, num_patches + 1, dim)) ##Generate the pos value'
        self.cls_token = nn.Parameter(torch.randn(1, 1, dim))                ##Generate the class value'
        self.dropout = nn.Dropout(emb_dropout)

        self.transformer = Transformer(dim, depth, heads, dim_head, mlp_dim, dropout)

        self.pool = pool
        self.to_latent = nn.Identity()

        self.mlp_head = nn.Sequential(
            nn.LayerNorm(dim),
            nn.Linear(dim, num_classes)
        )

    def forward(self, img):
        
        x = self.to_patch_embedding(img)
        b, n, _ = x.shape

        cls_tokens = repeat(self.cls_token, '() n d -> b n d', b = b)
        x = torch.cat((cls_tokens, x), dim=1)
        x += self.pos_embedding[:, :(n + 1)]
        x = self.dropout(x)

        x = self.transformer(x)

        x = x.mean(dim = 1) if self.pool == 'mean' else x[:, 0]

        x = self.to_latent(x)
        return self.mlp_head(x)


In [ ]:
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(DEVICE)

Transmodel = ViT(
    image_size = 30, 
    time_size = 1, 
    fre_size = 4, 
    num_classes = 5, 
    dim = 32, 
    depth = 8, 
    heads = 4, 
    mlp_dim = 128, 
    channels = 2,
    dim_head = 16
).to(DEVICE)


criterion = torch.nn.CrossEntropyLoss()
#optimizer = torch.optim.AdamW(Transmodel.parameters(), lr=1e-3)

#Transmodel.load_state_dict(torch.load('PreTrained_t_Trans_8lay_50drop_mulchan_eegspec_stateTrain'))
Transmodel.load_state_dict(torch.load('PreTest_Trans_8lay_50drop_noW_eegspec_state3'))



# Training and Validation

In [ ]:
optimizer = torch.optim.AdamW(Transmodel.parameters(), lr=1e-4) ##for test of pretrained
EPOCH = 150
loss_list = []
los_min = 10**10
val_loss_lis t= []
ac_list = []

for epoch in tqdm.tqdm(range(EPOCH)):
    running_loss = 0.0
    count = 0
    
    for _, (inputs, labels) in enumerate(train_dataloader, 0):

        optimizer.zero_grad()
        inputs = inputs.to(DEVICE)
        labels = labels.to(DEVICE)
        outputs = Transmodel(inputs)
        loss = criterion(outputs, labels.squeeze())
        loss.backward()
        optimizer.step()
        count = count+1

        # print statistics
        running_loss += loss.item()
        
    loss_loss=running_loss/count
    loss_list.append(loss_loss)
    print('epoch',epoch+1,':finished')
    print('train_loss:',loss_loss)
    
    with torch.no_grad():
        count=0
        running_loss=0.0
        pre=list()
        lab=list()
        
        for _, (inputs, labels) in enumerate(test_dataloader, 0):
            inputs=inputs.to(DEVICE)
            labels=labels.to(DEVICE)
            
            outputs = Transmodel(inputs)
            loss =criterion(outputs, labels.squeeze())
            running_loss += loss.item()
            count+=1
            _, predicted = torch.max(F.softmax(outputs).data, 1)
            predicted = predicted.to('cpu')
            labels = labels.to('cpu')
            
            predicted = predicted.tolist()
            labels = labels.tolist()
            pre.append(predicted)
            lab.append(labels)
            
        loss_loss = running_loss/count
        val_loss_list.append(loss_loss)
        pre = sum(pre,[])
        lab = sum(lab,[])
        
        print('val_loss:',loss_loss)
        cl = classification_report(lab, pre,output_dict=True)
        print(cl)
        ac_list.append(cl['accuracy'])
#         if los_min>loss_loss:
#             los_min=loss_loss
#             torch.save(Transmodel.state_dict(),'Trans_8lay_state') 

#        torch.save(Transmodel.state_dict(),'PreTrain_Trans_8lay_nodrop_mulchan_eegspec_state')
        torch.save(Transmodel.state_dict(),'PreTest_Trans_8lay_50drop_noW_eegspec_state6')

In [ ]:
import matplotlib.pyplot as plt
plt.plot(loss_list[1:])
plt.plot(val_loss_list[1:])

In [ ]:
loss_l = pd.DataFrame(loss_list[1:], columns=None)
val_loss_l = pd.DataFrame(val_loss_list[1:], columns=None)
ac_l = pd.DataFrame(ac_list, columns=None)
loss_l.to_csv('/home/is/zheng-ch/Transformer/PreTdat/PreTrained_t_Trans_8lay_50drop/PrenonWTestloss_lst6.csv', index=False)
val_loss_l.to_csv('/home/is/zheng-ch/Transformer/PreTdat/PreTrained_t_Trans_8lay_50drop/PrenonWTestval_loss_lst6.csv', index=False)
ac_l.to_csv('/home/is/zheng-ch/Transformer/PreTdat/PreTrained_t_Trans_8lay_50drop/PrenonWTestac_lst6.csv', index=False)


In [ ]:
plt.plot(ac_list)

In [32]:
EPOCH = 2
loss_list=[]
los_min=10**10
val_loss_list=[]
ac_list=[]

for epoch in tqdm.tqdm(range(EPOCH)):
    with torch.no_grad():
        count=0
        running_loss=0.0
        pre=list()
        lab=list()
        for _, (inputs, labels) in enumerate(test_dataloader, 0):
            inputs=inputs.to(DEVICE)
            labels=labels.to(DEVICE)
            outputs = Transmodel(inputs)
            loss =criterion(outputs, labels.squeeze())
            running_loss += loss.item()
            count+=1
            _, predicted = torch.max(F.softmax(outputs).data, 1)
            predicted=predicted.to('cpu')
            labels=labels.to('cpu')
            predicted=predicted.tolist()
            labels=labels.tolist()
            pre.append(predicted)
            lab.append(labels)
        loss_loss=running_loss/count
        val_loss_list.append(loss_loss)
        pre=sum(pre,[])
        lab=sum(lab,[])
        print('val_loss:',loss_loss)
        cl = classification_report(lab, pre,output_dict=True)
        print(cl)
        ac_list.append(cl['accuracy'])


  0%|          | 0/2 [00:00<?, ?it/s]/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


val_loss: 0.2842285505784895



 50%|█████     | 1/2 [02:15<02:15, 135.72s/it]

{'0': {'precision': 0.9643047360451151, 'recall': 0.9388946225844336, 'f1-score': 0.9514300507724053, 'support': 196693}, '1': {'precision': 0.603467178292449, 'recall': 0.33340904788945674, 'f1-score': 0.4295154722134166, 'support': 26415}, '2': {'precision': 0.890987787309704, 'recall': 0.912200161441949, 'f1-score': 0.901469205125722, 'support': 301037}, '3': {'precision': 0.9026397564933343, 'recall': 0.875841274414967, 'f1-score': 0.8890386132046401, 'support': 97174}, '4': {'precision': 0.8108771465877725, 'recall': 0.9061449521628311, 'f1-score': 0.85586811461468, 'support': 108284}, 'accuracy': 0.8927005508475157, 'macro avg': {'precision': 0.834455320945675, 'recall': 0.7932980116987276, 'f1-score': 0.8054642911861727, 'support': 729603}, 'weighted avg': {'precision': 0.8900059403928737, 'recall': 0.8927005508475157, 'f1-score': 0.8894277139773409, 'support': 729603}}
val_loss: 0.2842285505784895



100%|██████████| 2/2 [04:33<00:00, 136.61s/it]

{'0': {'precision': 0.9643047360451151, 'recall': 0.9388946225844336, 'f1-score': 0.9514300507724053, 'support': 196693}, '1': {'precision': 0.603467178292449, 'recall': 0.33340904788945674, 'f1-score': 0.4295154722134166, 'support': 26415}, '2': {'precision': 0.890987787309704, 'recall': 0.912200161441949, 'f1-score': 0.901469205125722, 'support': 301037}, '3': {'precision': 0.9026397564933343, 'recall': 0.875841274414967, 'f1-score': 0.8890386132046401, 'support': 97174}, '4': {'precision': 0.8108771465877725, 'recall': 0.9061449521628311, 'f1-score': 0.85586811461468, 'support': 108284}, 'accuracy': 0.8927005508475157, 'macro avg': {'precision': 0.834455320945675, 'recall': 0.7932980116987276, 'f1-score': 0.8054642911861727, 'support': 729603}, 'weighted avg': {'precision': 0.8900059403928737, 'recall': 0.8927005508475157, 'f1-score': 0.8894277139773409, 'support': 729603}}


# Visualization

In [ ]:
label.index([0])
indexes = [i for i,x in enumerate(label) if x == [0]]
print(indexes)

In [ ]:
def maka_patch_data(dat):
    dat=np.array(dat).reshape(-1,2,32,30)
    fixdata = dat[:,:,0:16,:]
    mean_p1 = np.mean(dat[:,:,16:20,:], axis = 2)
    mean_p2 = np.mean(dat[:,:,20:24,:], axis = 2)
    mean_p3 = np.mean(dat[:,:,24:28,:], axis = 2)
    mean_p4 = np.mean(dat[:,:,28:32,:], axis = 2)
    num_data = len(dat)
    ch = 2
    inputdat = np.concatenate((fixdata,mean_p1.reshape(num_data, ch, 1, 30),mean_p2.reshape(num_data, ch, 1, 30),mean_p3.reshape(num_data, ch, 1, 30),mean_p4.reshape(num_data, ch, 1, 30)),axis=2)
    return torch.tensor(inputdat)


class Mydatasets(torch.utils.data.Dataset):
    def __init__(self, data, label , make_patch=False,transform = None):
        self.transform = transform

        self.data = data
        self.label = label
        self.make_patch=make_patch

        self.datanum = len(data)

    def __len__(self):
        return self.datanum

    def __getitem__(self, idx):
        if not self.make_patch:
            out_data = torch.tensor(self.data[idx]).float()
            out_label = torch.tensor(self.label[idx])
        else:
            out_data = maka_patch_data(self.data[idx]).float()
            out_label = torch.tensor(self.label[idx])
        if self.transform:
            out_data = self.transform(out_data)

        return out_data, out_label

In [ ]:
print(dat.shape)
# label = index[0:100].values.tolist()#list
print(len(label))
fixdata = dat[:,:,0:16,:]
mean_p1 = np.mean(dat[:,:,16:20,:], axis = 2)
mean_p2 = np.mean(dat[:,:,20:24,:], axis = 2)
mean_p3 = np.mean(dat[:,:,24:28,:], axis = 2)
mean_p4 = np.mean(dat[:,:,28:32,:], axis = 2)
num_data = len(dat)
ch = 2
inputdat = np.concatenate((fixdata,mean_p1.reshape(num_data, ch, 1, 30),mean_p2.reshape(num_data, ch, 1, 30),mean_p3.reshape(num_data, ch, 1, 30),mean_p4.reshape(num_data, ch, 1, 30)),axis=2)
print(inputdat.shape)

train, test, train_label, test_label = train_test_split(dat, np.array(label),test_size = 0.1,stratify = label,random_state = 0)
print('train data:',len(train))
print('test data:',len(test))

train_data_set = Mydatasets(data=train,label=train_label)
test_data_set = Mydatasets(data=test,label=test_label,make_patch=False)
test_data_set_patch = Mydatasets(data=test,label=test_label,make_patch=True)


train_dataloader = torch.utils.data.DataLoader(train_data_set, batch_size = 64, shuffle=True)
test_dataloader = torch.utils.data.DataLoader(test_data_set, batch_size = 64, shuffle=False)

In [ ]:
##no train test split
print(dat.shape)
# label = index[0:100].values.tolist()#list
print(len(label))
fixdata = dat[:,:,0:16,:]
mean_p1 = np.mean(dat[:,:,16:20,:], axis = 2)
mean_p2 = np.mean(dat[:,:,20:24,:], axis = 2)
mean_p3 = np.mean(dat[:,:,24:28,:], axis = 2)
mean_p4 = np.mean(dat[:,:,28:32,:], axis = 2)
num_data = len(dat)
ch = 2

inputdat = np.concatenate((fixdata,mean_p1.reshape(num_data, ch, 1, 30),mean_p2.reshape(num_data, ch, 1, 30),mean_p3.reshape(num_data, ch, 1, 30),mean_p4.reshape(num_data, ch, 1, 30)),axis=2)
print(inputdat.shape)

test_data_set = Mydatasets(data=dat,label=np.array(label),make_patch=False)
test_data_set_patch = Mydatasets(data=dat,label=np.array(label),make_patch=True)



In [ ]:
##Visualization function 

class PreNorm(nn.Module): 
    def __init__(self, dim, fn):
        super().__init__()
        self.norm = nn.LayerNorm(dim)
        self.fn = fn
    def forward(self, x, **kwargs):
        return self.fn(self.norm(x), **kwargs)

class FeedForward(nn.Module):
    def __init__(self, dim, hidden_dim, dropout = 0.5):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(dim, hidden_dim),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, dim),
            nn.Dropout(dropout)
        )
    def forward(self, x):
        return self.net(x)

class Attention(nn.Module):
    def __init__(self, dim, heads, dim_head, dropout = 0.5):
        super().__init__()
        inner_dim = dim_head *  heads       ##32(4*8)   
        project_out = not (heads == 1 and dim_head == dim)

        self.heads = heads
        self.scale = dim_head ** -0.5

        self.attend = nn.Softmax(dim = -1)
        self.to_qkv = nn.Linear(dim, inner_dim * 3, bias = False)   ##better to dim to dim*3

        self.to_out = nn.Sequential(
            nn.Linear(inner_dim, dim),
            nn.Dropout(dropout)
        ) if project_out else nn.Identity()

    def forward(self, x):
        b, n, _, h = *x.shape, self.heads
        qkv = self.to_qkv(x).chunk(3, dim = -1)
        q, k, v = map(lambda t: rearrange(t, 'b n (h d) -> b h n d', h = h), qkv)

        dots = einsum('b h i d, b h j d -> b h i j', q, k) * self.scale

        attn = self.attend(dots)

        out = einsum('b h i j, b h j d -> b h i d', attn, v)
        out = rearrange(out, 'b h n d -> b n (h d)')
        return self.to_out(out),attn
        

class Transformer(nn.Module):           ##Register the blocks into whole network
    def __init__(self, dim, depth, heads, dim_head, mlp_dim, dropout = 0.5):
        super().__init__()
        self.layers = nn.ModuleList([])
        for _ in range(depth):
            self.layers.append(nn.ModuleList([
                PreNorm(dim, Attention(dim, heads = heads, dim_head = dim_head, dropout = dropout)),
                PreNorm(dim, FeedForward(dim, mlp_dim, dropout = dropout))
            ]))

    def forward(self, x):
        for attn, ff in self.layers:
            vdot,attention=attn(x)
            x = vdot + x
            x = ff(x) + x
        return x
    
    def get_attention(self,x):
        attention_list=[]
        for attn, ff in self.layers:
            vdot,attention=attn(x)
            attention_list.append(attention.cpu())
        return attention_list

class ViT(nn.Module):
    def __init__(self, *, image_size, time_size, fre_size, num_classes, dim, depth, heads, mlp_dim, pool = 'cls', channels , dim_head, dropout = 0.5, emb_dropout = 0.):
        super().__init__()
        assert image_size == 30  ##Time dimensions must equal to 30s
        num_patches = 300       #30*5'
        patch_dim = time_size * fre_size    #1*4'
        
        assert pool in {'cls', 'mean'}, 'pool type must be either cls (cls token) or mean (mean pooling)'

        self.to_patch_embedding = nn.Sequential(
            Rearrange('b c (h p1) (w p2) -> b (h w c) (p1 p2)', p1 = fre_size, p2 = time_size),
            nn.Linear(patch_dim, dim),        ##4 to dim
        )

        self.pos_embedding = nn.Parameter(torch.randn(1, num_patches + 1, dim)) ##Generate the pos value'
        self.cls_token = nn.Parameter(torch.randn(1, 1, dim))                ##Generate the class value'
        self.dropout = nn.Dropout(emb_dropout)

        self.transformer = Transformer(dim, depth, heads, dim_head, mlp_dim, dropout)

        self.pool = pool
        self.to_latent = nn.Identity()

        self.mlp_head = nn.Sequential(
            nn.LayerNorm(dim),
            nn.Linear(dim, num_classes)
        )

    def forward(self, img):
        x = self.to_patch_embedding(img)
        b, n, _ = x.shape

        cls_tokens = repeat(self.cls_token, '() n d -> b n d', b = b)
        x = torch.cat((cls_tokens, x), dim=1)
        x += self.pos_embedding[:, :(n + 1)]
        x = self.dropout(x)

        x = self.transformer(x)

        x = x.mean(dim = 1) if self.pool == 'mean' else x[:, 0]

        x = self.to_latent(x)
        return self.mlp_head(x)
    
    def get_attention(self,img):
        x = self.to_patch_embedding(img)
        b, n, _ = x.shape
        cls_tokens = repeat(self.cls_token, '() n d -> b n d', b = b)
        x = torch.cat((cls_tokens, x), dim=1)
        x += self.pos_embedding[:, :(n + 1)]
        attention = self.transformer.get_attention(x)
        return attention




In [ ]:
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(DEVICE)
criterion = torch.nn.CrossEntropyLoss()

### Patch visualization ####

Transmodel = ViT(
    image_size = 30, 
    time_size = 1, 
    fre_size = 4, 
    num_classes = 5, 
    dim = 32, 
    depth = 8, 
    heads = 4, 
    mlp_dim = 128, 
    channels = 2,
    dim_head = 16
)


Transmodel .load_state_dict(torch.load('/project/hikaku_db/_koki-od/eeg_vit/PreTest_Trans_8lay_50drop_noW_eegspec_state6'))
### Second-wise visualization ####
Transmodel_1s= ViT(
    image_size = 30, 
    time_size = 1,  ### 1 second
    fre_size = 32, ##### all bands
    num_classes = 5, 
    dim = 32, 
    depth = 8, 
    heads = 4, 
    mlp_dim = 128, 
    channels = 2,
    dim_head = 16
)
Transmodel_1s .load_state_dict(torch.load('/project/hikaku_db/_koki-od/eeg_vit/PreTest_Trans_8lay_nodrop_noW_eegspec_state6_1second'))
#optimizer = torch.optim.AdamW(Transmodel.parameters(), lr=1e-3)




In [ ]:
import cv2

def patch_attention(idx,dataset=test_data_set_patch):
    Transmodel.eval()
    
    data=dataset[idx][0]
    index = Transmodel(data.reshape(-1,2,20,30))
    att_mat = torch.stack(Transmodel.get_attention(data.reshape(-1,2,20,30))).squeeze(1)
    #print(att_mat.shape)
    att_mat = torch.mean(att_mat, dim=1)
    #print(att_mat.shape)
    residual_att = torch.eye(att_mat.size(1))
    aug_att_mat = att_mat + residual_att
    aug_att_mat = aug_att_mat / aug_att_mat.sum(dim=-1).unsqueeze(-1)
    #print(aug_att_mat.shape)
    joint_attentions = torch.zeros(aug_att_mat.size())
    joint_attentions[0] = aug_att_mat[0]
    
    for n in range(1, aug_att_mat.size(0)):
        joint_attentions[n] = torch.matmul(aug_att_mat[n], joint_attentions[n-1])
    
    for i, v in enumerate(joint_attentions[-1:]):
        # Attention from the output token to the input space.
        mask = v[0, 1:].detach().numpy()
        #print(mask.shape)
        img=np.squeeze(Rearrange('b (h w c) -> b c h w',h = 5, w = 30)(mask.reshape(-1,300)))
    
    return img,index

In [ ]:
def sec_attention(idx,dataset=test_data_set):
    Transmodel_1s.eval()
    data=test_data_set[idx][0]
    index=Transmodel_1s(data.reshape(-1,2,32,30))
    
    att_mat = torch.stack(Transmodel_1s.get_attention(data.reshape(-1,2,32,30))).squeeze(1)
    #print(att_mat.shape)
    att_mat = torch.mean(att_mat, dim=1)
    #print(att_mat.shape)
    residual_att = torch.eye(att_mat.size(1))
    aug_att_mat = att_mat + residual_att
    aug_att_mat = aug_att_mat / aug_att_mat.sum(dim=-1).unsqueeze(-1)
    #print(aug_att_mat.shape)
    joint_attentions = torch.zeros(aug_att_mat.size())
    joint_attentions[0] = aug_att_mat[0]
    
    for n in range(1, aug_att_mat.size(0)):
        joint_attentions[n] = torch.matmul(aug_att_mat[n], joint_attentions[n-1])
    
    for i, v in enumerate(joint_attentions[-1:]):
        # Attention from the output token to the input space.
        mask = v[0, 1:].detach().numpy()
        #print(mask.shape)
        img=np.squeeze(Rearrange('b (h w c) -> b c h w',h = 1, w = 30)(mask.reshape(-1,60)))
    
    return img,index,test_data_set[idx][1].item()

In [ ]:
def show_all(idx):
    img_sec,index,label=sec_attention(idx)
    print(label)
    print(torch.argmax(index).item())
    img_patch,index=patch_attention(idx)
    print(torch.argmax(index).item())
    
    fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(16, 4))
    ax1.set_title('Original_1ch')
    ax2.set_title('Original_2ch')
    original_img1= ax1.imshow(test_data_set[idx][0][0].numpy())
    fig.colorbar(original_img1, ax=ax1)
    original_img2 = ax2.imshow(test_data_set[idx][0][1].numpy())
    fig.colorbar(original_img2, ax=ax2)
    plt.show()
    
    fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(16, 4))
    img_sec0=cv2.resize(img_sec[0].reshape(1,30),(30,20))
    img_sec1=cv2.resize(img_sec[1].reshape(1,30),(30,20))
    ax1.set_title('attention1s_1ch')
    ax2.set_title('attention1s_2ch')
    sec_img1 = ax1.imshow(img_sec0)
    fig.colorbar(sec_img1, ax=ax1)
    sec_img2 = ax2.imshow(img_sec1)
    fig.colorbar(sec_img2, ax=ax2)
    plt.savefig('/home/is/zheng-ch/Transformer/resultvit/N2sameF1attenvit_1s.pdf')
    
    fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(16, 4))
    img_patch0=cv2.resize(img_patch[0].reshape(5,30),(30,20))
    img_patch1=cv2.resize(img_patch[1].reshape(5,30),(30,20))
    ax1.set_title('attention_patch_1ch')
    ax2.set_title('attention_patch_2ch')
    patch_img0 = ax1.imshow(img_patch0)
    fig.colorbar(patch_img0 , ax=ax1)
    #img_patch0.savefig("whole_figure.png")
    patch_img1 = ax2.imshow(img_patch1)
    fig.colorbar(patch_img1 , ax=ax2)
    #cv2.imwrite('ViTattentionc3.pdf',patch_img1)
    plt.savefig('/home/is/zheng-ch/Transformer/resultvit/N2sameF1attenvit_patch.pdf')
    

In [ ]:
ls=1033
show_all(ls)